In [5]:
import os
import re
os.chdir(r"C:\Users\local-admin\Desktop\Learning\09 - Python Projects\12 DoneDeal\00 Scripts\DoneDeal")

In [74]:
import donedeal as dd
from donedeal.constants import RECOMMENDED_COLUMNS
import pandas as pd

pd.options.display.max_columns = None

scraper = dd.CarScraper()
scraper.set_make_model(make = "Mercedes-Benz", model = "C-Class")
scraper.set_make_model(make='Volkswagen',model ='Golf')
scraper.set_area(area = "Dublin",countyTown="Ranelagh",radius=10)
scraper.set_price(price_from = 5000, price_to = 20000)
scraper.set_mileage(mileage_from = 10000, mileage_to = 140000)
scraper.set_year(year_from = 2015, year_to = 2024)
scraper.set_seller(seller="private")
scraper.url

'https://www.donedeal.ie/cars?make=Mercedes-Benz;model:C-Class&make=Volkswagen;model:Golf&area=Dublin&countyTown=Ranelagh&radius=10&price_from=5000&price_to=20000&mileage_from=10000&mileage_to=140000&year_from=2015&year_to=2024&sellerType=private'

In [75]:
scraper.scrape()
data = scraper.DataFrame.copy()[RECOMMENDED_COLUMNS]
data

retrieved 31 urls matching your criteria
retrieved 31 jsons
converted jsons to DataFrame


,make,model,price,fuelType,trim,transmission,engine,enginePower,acceleration,seats,numDoors,colour,country,owners,roadTax,NCTExpiry,bodyType,year,mileage,currency,sellerType,county,countyTown,geoPoint,header,description,friendlyUrl
0,Volkswagen,Golf,"18,200",Diesel,GTD,Manual,2.0 L,184 hp,7.5 sec,5,5,Black,Ireland,2,200,Mar 2026,Hatchback,2016,"73,000 mi",EUR,PRIVATE,Co. Dublin,South City,"53.324234,-6.265176",VOLKSWAGEN GOLF 2.0 GTD 184BHP NCT 03/26,VW Golf GTD 184HP \n71.000 miles\nNCT 03/26\nT...,https://www.donedeal.ie/cars-for-sale/volkswag...
1,Volkswagen,Golf,"14,950",Petrol,HIGHLINE,Automatic,1.4 L,140 hp,8.4 sec,5,3,Silver,Ireland,2,200,Mar 2025,Hatchback,2015,"105,000 km",EUR,PRIVATE,Co. Dublin,NaN,"53.366943,-6.340699",Volkswagen Golf *sunroof* *low kms*,"High spec, Excellent condition Volkswagen Golf...",https://www.donedeal.ie/cars-for-sale/volkswag...
2,Mercedes-Benz,C-Class,"16,000",Diesel,AVANTGARDE,Automatic,1.6 L,116 hp,11.6 sec,5,4,Blue,Ireland,2,200,Jul 2026,Saloon,2016,"66,700 km",EUR,PRIVATE,Co. Dublin,South County,"53.269971,-6.224923",Mercedes-Benz C-Class 2016 Automatic,Trade in Accept\nVehicle Features :\nMercedes-...,https://www.donedeal.ie/cars-for-sale/mercedes...
3,Volkswagen,Golf,"15,500",Petrol,GTE,Automatic,1.4 L,204 hp,7.6 sec,5,5,White,Ireland,1,200,Feb 2026,Hatchback,2015,"115,000 km",EUR,PRIVATE,Co. Dublin,Coolock,"53.39175486,-6.209078059",2015 golf tsi highline,VIEWING BY APPOINTMENT ONLY \n\n(Add can not b...,https://www.donedeal.ie/cars-for-sale/2015-gol...
4,Volkswagen,Golf,"15,900",Diesel,Highline,Manual,1.6 L,110 hp,10.7 sec,5,4,Blue,Ireland,1,190,May 2026,Estate,2016,"135,300 km",EUR,PRIVATE,Co. Dublin,NaN,"53.366943,-6.340699",Vw golf highLine 2016 TDI,Vw golf HighLine 1.6 TDI BlueMotion ...,https://www.donedeal.ie/cars-for-sale/vw-golf-...
5,Volkswagen,Golf,"15,950",Petrol,Trendline,Automatic,1.2 L,110 hp,9.9 sec,5,5,Grey,Ireland,3,200,Feb 2026,Hatchback,2016,"70,600 km",EUR,PRIVATE,Co. Dublin,NaN,"53.366943,-6.340699",Volkswagen Golf 2016 Automatic - DSG (NCT Passed),"Volkswagen Golf Hatchback 2016 DSG, clean and ...",https://www.donedeal.ie/cars-for-sale/volkswag...
6,Volkswagen,Golf,"11,000",Diesel,,Manual,1.6 L,,,2,3,White,Ireland,1,,,Van,2018,"93,000 km",EUR,PRIVATE,Co. Dublin,Artane,"53.38436683,-6.208335912",Car,Emaculate condition low mileage 1 owner,https://www.donedeal.ie/cars-for-sale/car/3750...
7,Volkswagen,Golf,"17,500",Petrol,Highline,Manual,1.2 L,110 hp,9.9 sec,5,5,Black,Ireland,2,200,Mar 2025,Estate,2017,"58,490 km",EUR,PRIVATE,Co. Dublin,Coolock,"53.39175486,-6.209078059",Volkswagen R Line Golf 2017,Well looked after manual 2017 Volkswagen Golf ...,https://www.donedeal.ie/cars-for-sale/volkswag...
8,Volkswagen,Golf,"13,950",Petrol,Lounge,Automatic,1.2 L,110 hp,9.9 sec,5,5,Blue,Ireland,3,200,Oct 2026,Hatchback,2015,"64,200 km",EUR,PRIVATE,Co. Dublin,North City,"53.370621,-6.253765",152 VW GOLF AUTO NCT 10/26,Vw Golf Automatic \n1.2 tsi petrol\nNev NCT 10...,https://www.donedeal.ie/cars-for-sale/152-vw-g...
9,Volkswagen,Golf,"16,000",Petrol,Highline,Manual,1.2 L,110 hp,9.9 sec,5,5,Silver,Ireland,5,200,Jun 2024,Estate,2016,"75,000 km",EUR,PRIVATE,Co. Dublin,City Centre,"53.34954,-6.260479",Golf tsi highline 2016,Car is in great condition\n1.4 tsi \nlow milea...,https://www.donedeal.ie/cars-for-sale/golf-tsi...


In [73]:
data.make.value_counts()

make
Volkswagen       51
Mercedes-Benz     4
Name: count, dtype: int64

In [12]:
# scraper.scrape(batch_size=1000)
# df = scraper.DataFrame
raw_df = pd.read_pickle("data.pkl")

In [17]:
df = raw_df[RECOMMENDED_COLUMNS].copy()
def assign_lat_lon(df):
    df['lat lon'.split()]=df.geoPoint.str.extract(r'([+-]?\d+\.\d+),\s*([+-]?\d+\.\d+)').rename(columns={0:'lat',1:'lon'}).astype(float)
    df.drop(columns='geoPoint',inplace=True)
    return df
def assign_mileage(df):
    df[['clockedDistance','unit']] = df.mileage.str.replace(r',','').str.extract(r'([,\d]*)\s*(km|mi)').rename(columns={0:'clockedDistance',1:'unit'})
    df['clockedDistance'] = df.clockedDistance.astype(float)
    # df['kilometers'] = df.apply(lambda row: row.clockedDistance if row.unit == 'km' else row.clockedDistance*1.60934,axis=1)
    df['kilometers'] = (df.unit == 'km')*df.clockedDistance + (df.unit == 'mi')*df.clockedDistance*1.60934
    df.drop(columns=['mileage','clockedDistance','unit'],inplace=True)
    return df
def price_to_float(df):
    df.price = df.price.apply(lambda x: re.sub(r'[^\d]','',x) if isinstance(x,str) else x).astype(float)
    return df
    
df = assign_lat_lon(df)
df = assign_mileage(df)
df = price_to_float(df)
df.plot(kind='scatter',x='lon',y='lat',backend='plotly',color='currency',width=1000,height=800)
# df.currency

In [51]:
df.price.loc[lambda _: _<300000].hist(backend='plotly',binwidth=1000)

TypeError: histogram() got an unexpected keyword argument 'binwidth'

In [69]:
df.loc[df.price == 123456789]
def isBsPrice(x:float)->bool:
    s = str(x)
    if s == 'nan':
        return False
    bullshit:bool=True
    for idx,ch in enumerate(re.sub(r'\.\d*','',s)):
        bullshit *= int(ch) == idx+1
    return (bullshit == 1)
isBsPrice(123456789)
df.loc[df.price.apply(isBsPrice)].price.value_counts()        

price
1234.0         719
1.0            218
123.0           78
12345.0         22
123456.0        16
12.0             6
1234567.0        6
123456789.0      2
Name: count, dtype: int64

price
<class 'float'>    90220
Name: count, dtype: int64